In [131]:
import pandas as pd
from tqdm.notebook import tqdm

In [214]:
meta = pd.read_csv('meta_30_embedding.csv')

In [215]:
meta.head()

,Unnamed: 0,title,asin,final_description,embedding,category,price
0,0,Lipton Yellow Label Tea (loose tea) - 450g,4639725043,lipton yellow label tea use only the finest te...,"[-0.36482325196266174, 0.8115509748458862, -0....","['Grocery & Gourmet Food', 'Beverages', 'Coffe...",$12.46
1,1,"Mae Ploy Green Curry Paste, 14 oz",9742356831,mae ploy thai green curry paste ingredients gr...,"[0.32072803378105164, 1.1068389415740967, 0.07...","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",NaN
2,2,WHOPPERS Candy (Chocolate Covered Malted Milk ...,B00005BPQ9,whoppers malted milk balls are the classic con...,"[-0.8029127717018127, 1.1512556076049805, 0.06...","['Grocery & Gourmet Food', 'Candy & Chocolate'...",NaN
3,3,"PowerBar Performance Energy Bar, Chocolate, 2....",B00006BN4U,100 dv energy releasing b vitamins 320mg elect...,"[0.32304733991622925, 1.552841067314148, -0.17...","['Grocery & Gourmet Food', 'Breakfast Foods', ...",$40.85
4,4,"Wilton Jimmies Rainbow Sprinkle Assortment, 3....",B00006FMLY,sprinkle your treats with sweet color using th...,"[-1.3537788391113281, 0.6959276795387268, -0.4...","['Grocery & Gourmet Food', 'Cooking & Baking',...",$10.41


## price 처리
- Price → 구간 나누기<br>
1:  최소값 ~ q1 이하<br>
2 : q1 초과 ~ q2 이하<br>
3 : q2 초과 ~ q3 이하<br>
4 : q3 초과 ~ 최대값

In [216]:
meta['price'] = meta['price'].fillna('0')

In [217]:
meta['price'] =  meta['price'].map(lambda x : x.strip("$"))

In [218]:
meta['price2'] = 0
for i in range(len(meta)):
    try : 
        meta.loc[i, 'price2'] = float(meta.loc[i, 'price'])
    except :
        meta.loc[i, 'price2'] = 0

In [219]:
meta = meta.astype({'price2':'float64'})

In [220]:
meta['price2'].describe()

count    6186.000000
mean       13.035571
std        21.407689
min         0.000000
25%         0.000000
50%         9.495000
75%        20.500000
max       999.000000
Name: price2, dtype: float64

In [221]:
meta['price'].isna().sum()

0

In [222]:
2570/len(meta)

0.41545425153572585

In [223]:
meta['price3'] = 1

meta.loc[(meta['price2'] > 0) & (meta['price2'] <= 9.495),'price3'] = 2
meta.loc[(meta['price2'] > 9.495) & (meta['price2'] <= 20.5),'price3'] = 3 
meta.loc[(meta['price2'] > 20.5),'price3'] = 4

## category 처리

In [54]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit_transform([set(['sci-fi', 'thriller']), set(['comedy'])])

array([[0, 1, 1],
       [1, 0, 0]])

In [57]:
list(mlb.classes_)

['comedy', 'sci-fi', 'thriller']

In [58]:
import ast

In [224]:
meta['category'] =  meta['category'].map(lambda x : ast.literal_eval(x))

In [225]:
meta.loc[1, 'category']

['Grocery & Gourmet Food',
 'Sauces, Gravies & Marinades',
 'Sauces',
 'Asian',
 'Curry Sauce']

In [226]:
mlb.fit_transform(meta['category'])

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [228]:
total = []
for i in range(6186):
    total = total + meta.loc[i, 'category']

In [77]:
len(total)

27169

In [234]:
from collections import Counter
counter = Counter(total)
totaldf = pd.DataFrame.from_dict(counter, orient='index').reset_index()
totaldf.columns = ['categoryword', 'count']
len(totaldf[totaldf['count']>=35])

97

In [239]:
len(totaldf)

730

In [235]:
totaldf[totaldf['categoryword']>='etc']

,categoryword,count


In [236]:
totaldf['new'] = totaldf['categoryword']
totaldf.loc[totaldf['count']<35, 'new'] = 'etc'

In [237]:
changeword = totaldf.loc[totaldf['new'] == 'etc', 'categoryword'].tolist()

In [240]:
len(changeword)

633

In [253]:
meta['category2'] = '0'
tot = []
for i in tqdm(range(6186)):
    targetlist = meta.loc[i, 'category']
    truelist = []
    for target in targetlist:
        if target in changeword:
            truelist.append('etc')
        else : truelist.append(target)
    truelist = str(list(set(truelist)))
    #print(truelist)
    meta.at[i, 'category2'] = truelist
    
meta['category2'] =  meta['category2'].map(lambda x : ast.literal_eval(x))

In [254]:
meta.loc[11, 'category']

['Grocery & Gourmet Food',
 'Herbs, Spices & Seasonings',
 'Mixed Spices & Seasonings',
 'Chinese Five Spice']

In [255]:
meta.loc[11, 'category2']

['Herbs, Spices & Seasonings',
 'Grocery & Gourmet Food',
 'etc',
 'Mixed Spices & Seasonings']

In [256]:
total = []
for i in range(6186):
    total = total + meta.loc[i, 'category2']

In [257]:
meta

,Unnamed: 0,title,asin,final_description,embedding,category,price,price2,price3,category2
0,0,Lipton Yellow Label Tea (loose tea) - 450g,4639725043,lipton yellow label tea use only the finest te...,"[-0.36482325196266174, 0.8115509748458862, -0....","[Grocery & Gourmet Food, Beverages, Coffee, Te...",12.46,12.46,3,"[Coffee, Tea & Cocoa, Grocery & Gourmet Food, ..."
1,1,"Mae Ploy Green Curry Paste, 14 oz",9742356831,mae ploy thai green curry paste ingredients gr...,"[0.32072803378105164, 1.1068389415740967, 0.07...","[Grocery & Gourmet Food, Sauces, Gravies & Mar...",0,0.00,1,"[etc, Sauces, Gravies & Marinades, Sauces, Gro..."
2,2,WHOPPERS Candy (Chocolate Covered Malted Milk ...,B00005BPQ9,whoppers malted milk balls are the classic con...,"[-0.8029127717018127, 1.1512556076049805, 0.06...","[Grocery & Gourmet Food, Candy & Chocolate, Ca...",0,0.00,1,"[Grocery & Gourmet Food, Candy & Chocolate Ass..."
3,3,"PowerBar Performance Energy Bar, Chocolate, 2....",B00006BN4U,100 dv energy releasing b vitamins 320mg elect...,"[0.32304733991622925, 1.552841067314148, -0.17...","[Grocery & Gourmet Food, Breakfast Foods, Brea...",40.85,40.85,4,"[Grocery & Gourmet Food, Breakfast & Cereal Ba..."
4,4,"Wilton Jimmies Rainbow Sprinkle Assortment, 3....",B00006FMLY,sprinkle your treats with sweet color using th...,"[-1.3537788391113281, 0.6959276795387268, -0.4...","[Grocery & Gourmet Food, Cooking & Baking, Fro...",10.41,10.41,3,"[Grocery & Gourmet Food, etc, Cooking & Baking]"
...,...,...,...,...,...,...,...,...,...,...
6181,6181,"Peanut Butter &amp; Co Bestie Almond Butter, N...",B01H3VFR6U,div class boost aplus container div class boo...,"[-0.8585600852966309, 1.205453872680664, -0.49...","[Grocery & Gourmet Food, Jams, Jellies & Sweet...",68.99,68.99,4,"[Nut & Seed Butters, Grocery & Gourmet Food, e..."
6182,6182,"Country Time Pink Lemonade Drink Mix, 5lb. 2.5oz.",B01H4G38JM,country time pink lemonade mix 82 5 ounce unit...,"[-1.002007246017456, 0.6213456392288208, -1.11...","[Grocery & Gourmet Food, Beverages, Bottled Be...",0,0.00,1,"[Soft Drink Mixes, Powdered Drink Mixes, Groce..."
6183,6183,"Kirkland Signature Supreme Whole Almonds, 2 Pa...",B01H6IQING,kirkland signature supreme whole almonds 2 pac...,"[0.38772085309028625, 1.314347267150879, 0.187...","[Grocery & Gourmet Food, Cooking & Baking, Nut...",38.98,38.98,4,"[Grocery & Gourmet Food, Cooking & Baking, Alm..."
6184,6184,"Kuli Kuli Moringa Green Energy Shots, Raspberr...",B01H6JO1EI,meet moringa the super green more nutritious t...,"[-0.7649469971656799, 0.6929209232330322, -0.9...","[Grocery & Gourmet Food, Breakfast Foods, Brea...",24.96,24.96,4,"[Grocery & Gourmet Food, Breakfast & Cereal Ba..."


In [265]:
cate_emb = mlb.fit_transform(meta['category2'])
meta = meta.assign(category_embedding = cate_emb.tolist())    

In [272]:
meta = meta.iloc[:, 1:]

In [275]:
meta.columns = ['title', 'asin', 'final_description', 'description_embedding', 
                'category', 'price','price2', 'final_price', 'category2', 'category_embedding']
meta = meta[['title', 'asin', 
             'final_description', 'description_embedding', 
             'price','price2', 'final_price', 
             'category','category2', 'category_embedding']]

In [276]:
meta.columns

Index(['title', 'asin', 'final_description', 'description_embedding', 'price',
       'price2', 'final_price', 'category', 'category2', 'category_embedding'],
      dtype='object')

In [277]:
meta.head()

,title,asin,final_description,description_embedding,price,price2,final_price,category,category2,category_embedding
0,Lipton Yellow Label Tea (loose tea) - 450g,4639725043,lipton yellow label tea use only the finest te...,"[-0.36482325196266174, 0.8115509748458862, -0....",12.46,12.46,3,"[Grocery & Gourmet Food, Beverages, Coffee, Te...","[Coffee, Tea & Cocoa, Grocery & Gourmet Food, ...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"Mae Ploy Green Curry Paste, 14 oz",9742356831,mae ploy thai green curry paste ingredients gr...,"[0.32072803378105164, 1.1068389415740967, 0.07...",0,0.00,1,"[Grocery & Gourmet Food, Sauces, Gravies & Mar...","[etc, Sauces, Gravies & Marinades, Sauces, Gro...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,WHOPPERS Candy (Chocolate Covered Malted Milk ...,B00005BPQ9,whoppers malted milk balls are the classic con...,"[-0.8029127717018127, 1.1512556076049805, 0.06...",0,0.00,1,"[Grocery & Gourmet Food, Candy & Chocolate, Ca...","[Grocery & Gourmet Food, Candy & Chocolate Ass...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ..."
3,"PowerBar Performance Energy Bar, Chocolate, 2....",B00006BN4U,100 dv energy releasing b vitamins 320mg elect...,"[0.32304733991622925, 1.552841067314148, -0.17...",40.85,40.85,4,"[Grocery & Gourmet Food, Breakfast Foods, Brea...","[Grocery & Gourmet Food, Breakfast & Cereal Ba...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
4,"Wilton Jimmies Rainbow Sprinkle Assortment, 3....",B00006FMLY,sprinkle your treats with sweet color using th...,"[-1.3537788391113281, 0.6959276795387268, -0.4...",10.41,10.41,3,"[Grocery & Gourmet Food, Cooking & Baking, Fro...","[Grocery & Gourmet Food, etc, Cooking & Baking]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [278]:
meta.to_csv('FINAL_META_DATA.csv', index=False)

In [259]:
totaldf

,categoryword,count,new
0,Grocery & Gourmet Food,6186,Grocery & Gourmet Food
1,Beverages,1203,Beverages
2,"Coffee, Tea & Cocoa",835,"Coffee, Tea & Cocoa"
3,Tea,349,Tea
4,Black,79,Black
...,...,...,...
725,Provolone,1,etc
726,Nuggets & Wings,1,etc
727,Fruit Bars,1,etc
728,Traditional Yogurt,1,etc


In [260]:
mlb.fit_transform(meta['category2']).shape

(6186, 98)

In [250]:
list(mlb.classes_)

['Active Dry Yeasts',
 'Adobo Sauce',
 'Adobo Seasoning',
 'Agave Nectar & Syrup',
 'Alfredo',
 'Allspice',
 'Almond',
 'Almond Butter',
 'Almond Milk',
 'Almonds',
 'American',
 'Anchovies',
 'Animal',
 'Anise Seeds',
 'Antipasto',
 'Apple Cider',
 'Apples',
 'Applesauce',
 'Applesauce & Fruit Cups',
 'Arborio',
 'Artichoke Hearts',
 'Asian',
 'Asian Dishes',
 'Asian Noodle Soups',
 'Asparagus',
 'Assortments & Samplers',
 'Avocado',
 'Avocados',
 'Bacon',
 'Bagel Chips',
 'Baked Beans',
 "Baker's Yeast",
 'Bakery & Dessert Gifts',
 'Baking Chocolates, Carob & Cocoa',
 'Baking Mixes',
 'Baking Powder',
 'Baking Soda',
 'Balsamic',
 'Bamboo Shoots',
 'Bananas & Plantains',
 'Barbecue',
 'Barbecue Seasoning',
 'Barley Flour',
 'Bars',
 'Basil',
 'Basmati',
 'Bay Leaf',
 'Bean',
 'Beans',
 'Beans & Peas',
 'Beef',
 'Beef Soups',
 'Beer Mustard',
 'Beets',
 'Bell Peppers',
 'Berries',
 'Beverages',
 'Biscotti',
 'Biscuits',
 'Bisques',
 'Bitters',
 'Black',
 'Black Beans',
 'Bloody Mary',

In [53]:
me

,Unnamed: 0,category,tech1,description,fit,title,also_buy,tech2,brand,feature,...,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details,price2,price3
0,10,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",NaN,['Lipton Yellow Label Tea use only the finest ...,NaN,Lipton Yellow Label Tea (loose tea) - 450g,"['B00886E4K0', 'B00CREXSHY', 'B001QTRGAQ', 'B0...",NaN,Lipton,[],...,Grocery,NaN,NaN,12.46,4639725043,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN,12.46,3
1,42,"['Grocery & Gourmet Food', 'Sauces, Gravies & ...",NaN,"['Mae Ploy Thai green curry paste.', 'Ingredie...",NaN,"Mae Ploy Green Curry Paste, 14 oz","['B000EICJWA', 'B000EICISA', 'B074ZMGQNT', 'B0...",NaN,Mae Ploy,[],...,Grocery,NaN,NaN,0,9742356831,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,{'\n Product Dimensions: \n ': '3.6 x 3....,0.00,1
2,53,"['Grocery & Gourmet Food', 'Candy & Chocolate'...",NaN,['WHOPPERS Malted Milk Balls are the classic c...,NaN,WHOPPERS Candy (Chocolate Covered Malted Milk ...,"['B0029JVUAE', 'B01EFCQDPE', 'B003WP05C8', 'B0...",NaN,Whoppers,"['<span class=""a-size-base a-color-secondary"">...",...,Grocery,NaN,NaN,0,B00005BPQ9,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,"{'Shipping Weight:': '5.8 pounds', 'Domestic S...",0.00,1
3,65,"['Grocery & Gourmet Food', 'Breakfast Foods', ...",NaN,['100% DV energy releasing B Vitamins. 320mg E...,NaN,"PowerBar Performance Energy Bar, Chocolate, 2....","['B00E0DRMTU', 'B00E0DT2NY', 'B0017RJERA', 'B0...",NaN,Powerbar,[],...,Grocery,NaN,NaN,40.85,B00006BN4U,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,"{'Shipping Weight:': '3.6 pounds (', 'Domestic...",40.85,4
4,67,"['Grocery & Gourmet Food', 'Cooking & Baking',...",NaN,['Sprinkle your treats with sweet color using ...,NaN,"Wilton Jimmies Rainbow Sprinkle Assortment, 3....","['B00IE72BXA', 'B007SQ70DY', 'B00IE72CEI', 'B0...",NaN,Wilton,['Bright jimmies add a burst of color to your ...,...,Amazon Home,NaN,"September 9, 2003",10.41,B00006FMLY,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,{},10.41,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6781,286268,"['Grocery & Gourmet Food', 'Cooking & Baking',...",NaN,"['Kirkland Signature Supreme Whole Almonds, 2 ...",NaN,"Kirkland Signature Supreme Whole Almonds, 2 Pa...","['B000XBCBW6', 'B00OPQZJA6', 'B0044MNYIK', 'B0...",NaN,Kirkland Signature,[],...,Grocery,NaN,NaN,38.98,B01H6IQING,[],[],"{'\n Item Weight: \n ': '6 pounds', 'Shi...",38.98,4
6782,286270,"['Grocery & Gourmet Food', 'Breakfast Foods', ...",NaN,['MEET MORINGA: The Super Green more nutritiou...,NaN,"Kuli Kuli Moringa Green Energy Shots, Raspberr...","['B00IJ2FAFQ', 'B0743D3BZJ', 'B007XW2Y8O', 'B0...",NaN,Kuli Kuli,"['<span class=""a-size-base a-color-secondary"">...",...,Grocery,NaN,NaN,24.96,B01H6JO1EI,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,"{'\n Item Weight: \n ': '2.1 pounds', 'S...",24.96,4
6783,286633,"['Grocery & Gourmet Food', 'Snack Foods', 'Jer...",NaN,"[""Grass Run Farms 100% Grass-Fed Beef Sticks p...",NaN,Grass Run Farms 100% Grass-Fed Gluten-Free- No...,"['B01BTD64IS', 'B00QU79XVE', 'B0117PMHO8', 'B0...",NaN,Grass Run Farms,[],...,Grocery,NaN,NaN,0,B01HCV88MQ,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,"{'\n Item Weight: \n ': '0.96 ounces', '...",0.00,1
6784,286835,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",NaN,[],NaN,Nescafe Taster's Choice Instant Coffee Beverag...,"['B01HGBAJEC', 'B01H3KK5H2', 'B01IC1M8U8', 'B0...",NaN,Taster's Choice,[],...,Grocery,NaN,NaN,0,B01HGBAFNC,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,"{'Shi